In [45]:
#pip install ucimlrepo

  Obtaining dependency information for ucimlrepo from https://files.pythonhosted.org/packages/3e/4a/ecc3456479d687202b34ee42317c3a63e09793c9409a720052d38356431a/ucimlrepo-0.0.3-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from ucimlrepo import fetch_ucirepo 

In [2]:
# Fetch dataset 
adult = fetch_ucirepo(id=2) 
target = adult.data.targets 

In [53]:
# Convert dictionary to DataFrame
df = pd.DataFrame(adult['data']['features'])
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


### Column Names

In [54]:
list(df.columns)

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country']

### Add `income` column as our target variable

In [55]:
df['income'] = target
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Dealing with `fnlwgt` column
- We normalise the weights (rather than creating all of the duplicates because that will increase time to train our models)

In [56]:
# Calculate total weight
total_weight = sum(list(df['fnlwgt']))

In [57]:
# Normalise the weights
df['fnlwgt'] = df['fnlwgt']/total_weight

### Rename `income` entries to `0` and `1`

In [58]:
# Mapping dictionary
mapping = {'<=50K': 0, '<=50K.': 0, '>50K': 1, '>50K.': 1}

# Relabel the values in the 'column_name' column
df['income'] = df['income'].map(mapping)

### Dealing with categorical variables

In [59]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [60]:
df.dtypes

age                 int64
workclass          object
fnlwgt            float64
education          object
education-num       int64
marital-status     object
occupation         object
relationship       object
race               object
sex                object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
income              int64
dtype: object

# Note: feel free to alter how we pre-process the data

### Missing values
- `workclass` has `NaN` and `?`
- `occupation` has `NaN` and `?`
- `native-country` has `NaN` and `?`

In [61]:
len(df[df.isnull().any(axis=1)]) + len(df[df.eq('?').any(axis=1)])

3620

### So there are 3,620 rows with either `NaN` or `?` 
- This is ~7.5% of the dataset, and will leave us with > 45,000 rows of clean data if we simply remove all of these rows

In [62]:
# Remove rows with NaN values
df = df.dropna()

# Remove rows with "?" entries
df = df[~df.eq('?').any(axis=1)]

### Deal with categorical variables

In [63]:
df = pd.get_dummies(df, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', \
                                 'race', 'sex', 'native-country'])

In [64]:
# Convert True/False to 1/0
df = df * 1

In [65]:
# Move income column back to the end
label_col = df.pop('income')
df['income'] = label_col

### Export DataFrame to a .csv file

In [68]:
df.to_csv('../Data/one_hot_data.csv', index=False)